# Training a convnet from scratch

### load the dataset

In [9]:
import os

In [33]:
original_data_cat = 'kagglecatsanddogs_5340/PetImages/Cat'

In [34]:
original_data_dog = 'kagglecatsanddogs_5340/PetImages/Dog'

In [23]:
base_dir = 'dogs-vs-cats-small'
os.mkdir(base_dir)

In [13]:
train_dir = os.path.join(base_dir, 'train')
os.mkdir(train_dir)

In [14]:
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)

In [15]:
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

In [16]:
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

In [17]:
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

In [18]:
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

In [19]:
import shutil

In [35]:
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_data_cat, fname) 
    dst = os.path.join(train_cats_dir, fname) 
    shutil.copyfile(src, dst)

In [36]:
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_data_cat, fname) 
    dst = os.path.join(validation_cats_dir, fname) 
    shutil.copyfile(src, dst)

In [37]:
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_data_cat, fname) 
    dst = os.path.join(test_cats_dir, fname) 
    shutil.copyfile(src, dst)

In [38]:
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_data_dog, fname) 
    dst = os.path.join(train_dogs_dir, fname) 
    shutil.copyfile(src, dst)

In [39]:
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_data_dog, fname) 
    dst = os.path.join(validation_dogs_dir, fname) 
    shutil.copyfile(src, dst)

In [40]:
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_data_dog, fname) 
    dst = os.path.join(test_dogs_dir, fname) 
    shutil.copyfile(src, dst)

In [45]:
print('total train cat images: ', len(os.listdir(train_cats_dir)))
print('total validation cat images: ',len(os.listdir(validation_cats_dir)))
print('total test cat images: ', len(os.listdir(test_cats_dir)))

total train cat images:  1000
total validation cat images:  500
total test cat images:  500


In [46]:
print('total train dog images: ', len(os.listdir(train_dogs_dir)))
print('total validation dog images: ',len(os.listdir(validation_dogs_dir)))
print('total test dog images: ', len(os.listdir(test_dogs_dir)))

total train dog images:  1000
total validation dog images:  500
total test dog images:  500


### build the network

In [48]:
from tensorflow.keras import layers, models

In [51]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       147584    
__________

### Configuring the model for training

In [52]:
from tensorflow.keras import optimizers

In [61]:
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4) , metrics=['acc'])

### Using ImageDataGenerator to read images from directories

In [63]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [64]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [65]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

Found 2000 images belonging to 2 classes.


In [67]:
validation_generator = test_datagen.flow_from_directory(validation_dir, target_size=(150, 150), batch_size=20, class_mode='binary')

Found 1000 images belonging to 2 classes.


In [69]:
for train_batch, label_batch in train_generator:
    print('train batch shape: ', train_batch.shape)
    print('label batch shape: ', label_batch.shape)
    break


train batch shape:  (20, 150, 150, 3)
label batch shape:  (20,)


In [89]:
history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=30, validation_data=validation_generator, validation_steps=50) 

Epoch 1/30
 86/100 [========================>.....] - ETA: 6s - loss: 0.4323 - acc: 0.7971

RuntimeError: generator raised StopIteration